#### Code reference for this section is from:
* https://jfh.georgetown.domains/dsan5000/slides-and-labs/_site/content/labs/code-demos/API-wikipedia/wikipedia-api.html
* https://jfh.georgetown.domains/dsan5000/slides-and-labs/_site/content/labs/code-demos/API-newapi/news-api.html


In [2]:
API_KEY='481b1e4a75874d2f9a23e3329031364c'

In [3]:
import requests
import json
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

### Topic : Electric Vehicles


In [4]:
baseURL = "https://newsapi.org/v2/everything?"
total_requests=2
verbose=True

# THIS CODE WILL NOT WORK UNLESS YOU INSERT YOUR API KEY IN THE NEXT LINE
API_KEY='481b1e4a75874d2f9a23e3329031364c'
TOPIC1='electric vehicles'

In [5]:
URLpost = {'apiKey': API_KEY,
            'q': '+'+TOPIC1,
            'sortBy': 'relevancy',
            'totalRequests': 1}

print(baseURL)
# print(URLpost)

#GET DATA FROM API
response = requests.get(baseURL, URLpost) #request data from the server
# print(response.url);  
response = response.json() #extract txt data from request into json

# PRETTY PRINT
# https://www.digitalocean.com/community/tutorials/python-pretty-print-json

print(json.dumps(response, indent=2))

# #GET TIMESTAMP FOR PULL REQUEST
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d-H%H-M%M-S%S")

# SAVE TO FILE 
with open(timestamp+'-newapi-raw-data.json', 'w') as outfile:
    json.dump(response, outfile, indent=4)

https://newsapi.org/v2/everything?
{
  "status": "ok",
  "totalResults": 7005,
  "articles": [
    {
      "source": {
        "id": "the-verge",
        "name": "The Verge"
      },
      "author": "Andrew J. Hawkins",
      "title": "Google wants to be at the center of all your climate change decisions",
      "description": "Google is rolling out a host of new updates aimed at helping people buy electric vehicles and heat pumps, and cities better plan for climate disasters.",
      "url": "https://www.theverge.com/2023/10/10/23906496/google-ai-transportation-ev-heat-pump-flood-wildfire",
      "urlToImage": "https://cdn.vox-cdn.com/thumbor/EaScurH_lVme4X1artA78Qh3lpo=/0x0:2040x1360/1200x628/filters:focal(1020x680:1021x681)/cdn.vox-cdn.com/uploads/chorus_asset/file/24016888/STK093_Google_01.jpg",
      "publishedAt": "2023-10-10T11:00:00Z",
      "content": "Google wants to be at the center of all your climate change decisions\r\nGoogle wants to be at the center of all your climate c

In [6]:
def string_cleaner(input_string):
    try: 
        out=re.sub(r"""
                    [,.;@#?!&$-]+  # Accept one or more copies of punctuation
                    \ *           # plus zero or more copies of a space,
                    """,
                    " ",          # and replace it with a single space
                    input_string, flags=re.VERBOSE)

        #REPLACE SELECT CHARACTERS WITH NOTHING
        out = re.sub('[’.]+', '', input_string)

        #ELIMINATE DUPLICATE WHITESPACES USING WILDCARDS
        out = re.sub(r'\s+', ' ', out)

        #CONVERT TO LOWER CASE
        out=out.lower()
    except:
        print("ERROR")
        out=''
    return out

In [7]:
article_list=response['articles']   #list of dictionaries for each article
article_keys=article_list[0].keys()
print("AVAILABLE KEYS:")
print(article_keys)
index=0
electric_vehicle_cleaned_data=[];  
for article in article_list:
    tmp=[]
    if(verbose):
        print("#------------------------------------------")
        print("#",index)
        print("#------------------------------------------")

    for key in article_keys:
        if(verbose):
            print("----------------")
            print(key)
            print(article[key])
            print("----------------")

        if(key=='source'):
            src=string_cleaner(article[key]['name'])
            tmp.append(src) 

        if(key=='author'):
            author=string_cleaner(article[key])
            #ERROR CHECK (SOMETIMES AUTHOR IS SAME AS PUBLICATION)
            if(src in author): 
                print(" AUTHOR ERROR:",author);author='NA'
            tmp.append(author)

        if(key=='title'):
            tmp.append(string_cleaner(article[key]))

        # if(key=='description'):
        #     tmp.append(string_cleaner(article[key]))

        # if(key=='content'):
        #     tmp.append(string_cleaner(article[key]))

        if(key=='publishedAt'):
            #DEFINE DATA PATERN FOR RE TO CHECK  .* --> wildcard
            ref = re.compile('.*-.*-.*T.*:.*:.*Z')
            date=article[key]
            if(not ref.match(date)):
                print(" DATE ERROR:",date); date="NA"
            tmp.append(date)

    electric_vehicle_cleaned_data.append(tmp)
    index+=1

AVAILABLE KEYS:
dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])
#------------------------------------------
# 0
#------------------------------------------
----------------
source
{'id': 'the-verge', 'name': 'The Verge'}
----------------
----------------
author
Andrew J. Hawkins
----------------
----------------
title
Google wants to be at the center of all your climate change decisions
----------------
----------------
description
Google is rolling out a host of new updates aimed at helping people buy electric vehicles and heat pumps, and cities better plan for climate disasters.
----------------
----------------
url
https://www.theverge.com/2023/10/10/23906496/google-ai-transportation-ev-heat-pump-flood-wildfire
----------------
----------------
urlToImage
https://cdn.vox-cdn.com/thumbor/EaScurH_lVme4X1artA78Qh3lpo=/0x0:2040x1360/1200x628/filters:focal(1020x680:1021x681)/cdn.vox-cdn.com/uploads/chorus_asset/file/24016888/STK093_G

In [8]:
df = pd.DataFrame(electric_vehicle_cleaned_data)
print(df)
df.to_csv('electric_vehicle_cleaned.csv', index=False) 
index_label=['title','src','author','date','description']

               0                  1  \
0      the verge   andrew j hawkins   
1          wired   julian chokkattu   
2       engadget      lawrence bonk   
3      the verge   andrew j hawkins   
4     gizmodocom       kevin hurler   
..           ...                ...   
95      autoblog     steven j ewing   
96   marketwatch       tomi kilgore   
97  ars technica  jonathan m gitlin   
98      autoblog            insider   
99      autoblog           engadget   

                                                    2                     3  
0   google wants to be at the center of all your c...  2023-10-10T11:00:00Z  
1   infinite machine p1 electric scooter: specs, r...  2023-10-12T17:15:00Z  
2   stockholm bans most combustion engine cars fro...  2023-10-12T17:32:40Z  
3   gms plan to make more electric trucks keeps ru...  2023-10-17T19:04:58Z  
4    ford, hertz pump brakes on ev plans, reports say  2023-10-27T18:00:00Z  
..                                                ...          

In [9]:
electric_vehicle_df = pd.read_csv("electric_vehicle_cleaned.csv")
rename_map = {
    '0': 'title',
    '2': 'description'
}

electric_vehicle_df.rename(columns=rename_map, inplace=True)

In [10]:
cols_to_keep = rename_map.values()
electric_vehicle_df = electric_vehicle_df[cols_to_keep]

In [11]:
electric_vehicle_df

title                                        description
0      the verge  google wants to be at the center of all your c...
1          wired  infinite machine p1 electric scooter: specs, r...
2       engadget  stockholm bans most combustion engine cars fro...
3      the verge  gms plan to make more electric trucks keeps ru...
4     gizmodocom   ford, hertz pump brakes on ev plans, reports say
..           ...                                                ...
95      autoblog  kia ev3 small suv concept is ‘very close to pr...
96   marketwatch  : nios stock rises toward 5-day win streak as ...
97  ars technica  the 2023 toyota prius prime is a mostly pleasa...
98      autoblog  50 new evs are hitting the us soon it's not cl...
99      autoblog      hertz is putting the brakes on 100,000 teslas

[100 rows x 2 columns]

In [12]:
import wikipedia

In [13]:
ev_text = str(electric_vehicle_df["description"])

In [14]:
print(ev_text)

0     google wants to be at the center of all your c...
1     infinite machine p1 electric scooter: specs, r...
2     stockholm bans most combustion engine cars fro...
3     gms plan to make more electric trucks keeps ru...
4      ford, hertz pump brakes on ev plans, reports say
                            ...                        
95    kia ev3 small suv concept is ‘very close to pr...
96    : nios stock rises toward 5-day win streak as ...
97    the 2023 toyota prius prime is a mostly pleasa...
98    50 new evs are hitting the us soon it's not cl...
99        hertz is putting the brakes on 100,000 teslas
Name: description, Length: 100, dtype: object


In [15]:
# MODIFIED FROM 
# https://towardsdatascience.com/simple-wordcloud-in-python-2ae54a9f58e5
def generate_word_cloud(my_text):
    from wordcloud import WordCloud, STOPWORDS
    import matplotlib.pyplot as plt
    # exit()
    # Import package
    # Define a function to plot word cloud
    def plot_cloud(wordcloud):
        # Set figure size
        plt.figure(figsize=(40, 30))
        # Display image
        plt.imshow(wordcloud) 
        # No axis details
        plt.axis("off");

    # Generate word cloud
    wordcloud = WordCloud(
        width = 3000,
        height = 2000, 
        random_state=1, 
        background_color='salmon', 
        colormap='Pastel1', 
        collocations=False,
        stopwords = STOPWORDS).generate(my_text)
    plot_cloud(wordcloud)
    plt.show()

# text='The field of machine learning is typically divided into three fundamental sub-paradigms. These include supervised learning, unsupervised learning, and reinforcement learning (RL). The discipline of reinforcement learning focuses on how intelligent agents learn to perform actions, inside a specified environment, to maximize  a cumulative reward function. Over the past several decades, there has been a push to incorporate concepts from the field of deep-learning into the agents used in RL algorithms. This has spawned the field of Deep reinforcement learning. To date, the field of deep RL has yielded stunning results in a wide range of technological applications. These include, but are not limited to, self-driving cars, autonomous game play, robotics, trading and finance, and Natural Language Processing. This course will begin with an introduction to the fundamentals of traditional, i.e. non-deep, reinforcement learning. After reviewing fundamental deep learning topics the course will transition to deep RL by incorporating artificial neural networks into the models. Topics include Markov Decision Processes, Multi-armed Bandits, Monte Carlo Methods, Temporal Difference Learning, Function Approximation, Deep Neural Networks, Actor-Critic, Deep Q-Learning, Policy Gradient Methods, and connections to Psychology and to Neuroscience.'
generate_word_cloud(ev_text)

### Data from API:

In [16]:
# # Code source: https://www.programiz.com/python-programming/json

# import json

# with open('output.json', 'r') as f:
#   data = json.load(f)

# str_json = json.dumps(data)

# # Getting dictionary
# json_dict = json.loads(str_json)

# # Pretty Printing JSON string back
# data_output = json.dumps(json_dict, indent = 4, sort_keys=True)
# print(data_output)

# import pandas as pd

# # Taking a subset now to test and later cleaning can be performed on the whole file
# data_subset = {
#     "2020 Census Tract": 53061042003,
#     "Base MSRP": 0,
#     "City": "Edmonds",
#     "Clean Alternative Fuel Vehicle (CAFV) Eligibility": "Clean Alternative Fuel Vehicle Eligible",
#     "County": "Snohomish",
#     "DOL Vehicle ID": 167322298,
#     "Electric Range": 124,
#     "Electric Utility": "PUGET SOUND ENERGY INC",
#     "Electric Vehicle Type": "Battery Electric Vehicle (BEV)",
#     "Legislative District": 21,
#     "Make": "HYUNDAI",
#     "Model": "IONIQ",
#     "Model Year": 2019,
#     "Postal Code": 98026,
#     "State": "WA",
#     "VIN (1-10)": "KMHC05LH6K",
#     "Vehicle Location": "POINT (-122.33568499999996 47.803720000000055)",
#     "_id": "64b8150c3f35c83376286f52"
# }

# # Create a Pandas DataFrame from the provided JSON data
# df = pd.json_normalize(data)

# df.rename(columns=lambda x: x.strip(), inplace=True)
# df=df.rename(columns = {'_id':'id'})
# df=df.rename(columns = {'VIN (1-10)':'VIN'})

# # Drop columns that are not needed:
# df = df.drop(['Base MSRP', 'Legislative District', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility', '2020 Census Tract'], axis=1)

# # Print the DataFrame
# display(df)

In [20]:
import json
import pandas as pd

# Load JSON file
with open('output.json', 'r') as f:
    data = json.load(f)

# Create a Pandas DataFrame from the entire JSON data
df = pd.json_normalize(data)

# Renaming columns and stripping any spaces
df.rename(columns=lambda x: x.strip(), inplace=True)
df=df.rename(columns = {'_id':'id'})
df=df.rename(columns = {'VIN (1-10)':'VIN'})

# Drop columns that are not needed:
# df = df.drop(['Base MSRP', 'Legislative District', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility', '2020 Census Tract'], axis=1)

# Print the DataFrame
display(df)

# saving the dataframe
df.to_csv('ev-output.csv')


id         VIN     County          City State  \
0   64b8150c3f35c83376286f21  1N4AZ0CP5D     Kitsap     Bremerton    WA   
1   64b8150c3f35c83376286f22  1N4AZ1CP8K     Kitsap  Port Orchard    WA   
2   64b8150c3f35c83376286f23  5YJXCAE28L       King       Seattle    WA   
3   64b8150c3f35c83376286f24  SADHC2S1XK   Thurston       Olympia    WA   
4   64b8150c3f35c83376286f25  JN1AZ0CP9B  Snohomish       Everett    WA   
..                       ...         ...        ...           ...   ...   
95  64b8150c3f35c83376286f80  1N4AZ0CP8G       King      Bellevue    WA   
96  64b8150c3f35c83376286f81  5UXKT0C53J     Yakima        Zillah    WA   
97  64b8150c3f35c83376286f82  5YJSA1H23F  Snohomish      Lynnwood    WA   
98  64b8150c3f35c83376286f83  5YJSA1E21J     Skagit     Anacortes    WA   
99  64b8150c3f35c83376286f84  1G6RL1E40G   Thurston     Rochester    WA   

    Postal Code  Model Year      Make    Model  \
0         98310        2013    NISSAN     LEAF   
1         98366        2019    NISSAN     LEAF   
2         98199        2020     TESLA  MODEL X   
3         98503        2019    JAGUAR   I-PACE   
4         98204        2011    NISSAN     LEAF   
..          ...         ...       ...      ...   
95        98006        2016    NISSAN     LEAF   
96        98953        2018       BMW       X5   
97        98036        2015     TESLA  MODEL S   
98        98221        2018     TESLA  MODEL S   
99        98579        2016  CADILLAC      ELR   

                     Electric Vehicle Type  \
0           Battery Electric Vehicle (BEV)   
1           Battery Electric Vehicle (BEV)   
2           Battery Electric Vehicle (BEV)   
3           Battery Electric Vehicle (BEV)   
4           Battery Electric Vehicle (BEV)   
..                                     ...   
95          Battery Electric Vehicle (BEV)   
96  Plug-in Hybrid Electric Vehicle (PHEV)   
97          Battery Electric Vehicle (BEV)   
98          Battery Electric Vehicle (BEV)   
99  Plug-in Hybrid Electric Vehicle (PHEV)   

   Clean Alternative Fuel Vehicle (CAFV) Eligibility  Electric Range  \
0            Clean Alternative Fuel Vehicle Eligible              75   
1            Clean Alternative Fuel Vehicle Eligible             150   
2            Clean Alternative Fuel Vehicle Eligible             293   
3            Clean Alternative Fuel Vehicle Eligible             234   
4            Clean Alternative Fuel Vehicle Eligible              73   
..                                               ...             ...   
95           Clean Alternative Fuel Vehicle Eligible              84   
96             Not eligible due to low battery range              13   
97           Clean Alternative Fuel Vehicle Eligible             208   
98           Clean Alternative Fuel Vehicle Eligible             249   
99           Clean Alternative Fuel Vehicle Eligible              40   

    Base MSRP  Legislative District  DOL Vehicle ID  \
0           0                    23       214384901   
1           0                    26       271008636   
2           0                    36         8781552   
3           0                     2         8308492   
4           0                    21       245524527   
..        ...                   ...             ...   
95          0                    41           35325   
96          0                    15       153996167   
97          0                     1         7821215   
98          0                    40       285304512   
99          0                    35       161629142   

                                  Vehicle Location  \
0   POINT (-122.61136499999998 47.575195000000065)   
1    POINT (-122.63926499999997 47.53730000000007)   
2            POINT (-122.394185 47.63919500000003)   
3                       POINT (-122.8285 47.03646)   
4    POINT (-122.24128499999995 47.91088000000008)   
..                                             ...   
95  POINT (-122.16936999999996 47.571015000000045)   
96   POINT (-